# # 필요한 라이브러리

In [1]:
import pandas as pd

## ## 구글 드라이브 마운트

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## ## 이미지 불러오기

In [3]:
from IPython.display import Image

# # 설치

In [4]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 6.4 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633713 sha256=2f815d192694c2f33f1002dab473504fa41872523819422dd9cb1fdab7b6422a
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [5]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

# # 데이터

In [6]:
data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0) 

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


## ## 데이터 분석
  * 사용자ID, 
  * 상품ID, 
  * 평점, 
  * 시간

In [7]:
df = pd.DataFrame(data.raw_ratings, columns=["user", "item", "rate", "id"])
df.head(100), df.shape

(   user  item  rate         id
 0   196   242   3.0  881250949
 1   186   302   3.0  891717742
 2    22   377   1.0  878887116
 3   244    51   2.0  880606923
 4   166   346   1.0  886397596
 ..  ...   ...   ...        ...
 95  119  1153   5.0  874781198
 96   62   498   4.0  879373848
 97   62   382   3.0  879375537
 98   28   209   4.0  881961214
 99  135    23   4.0  879857765
 
 [100 rows x 4 columns], (100000, 4))

In [8]:
df[df['user'] == '196']

,user,item,rate,id
0,196,242,3.0,881250949
940,196,393,4.0,881251863
1133,196,381,4.0,881251728
1812,196,251,3.0,881251274
1896,196,655,5.0,881251793
2374,196,67,5.0,881252017
6910,196,306,4.0,881251021
7517,196,238,4.0,881251820
7842,196,663,5.0,881251911
10017,196,111,4.0,881251793


# # 알고리즘 생성

In [9]:
algo = SVD(random_state=0)
algo.fit(trainset) 

# # 추천 수행
  * test()
    - 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측
    - 입력된 데이터 세트에 대해 추천 데이터 세트를 만든다.
  * predict()
    - 개별 사용자와 영화에 대한 추천 평점을 반환

## ## test()
  * uid : 개별 사용자 아이디
  * iid : 영화(아이템) 아이디
  * r_ui : 실제 평점
  * est : 예측 평점
  * was_impossible 
    - True : 예측값을 생성할 수 없는 데이터

In [10]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

### ### 해석


In [11]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

## ## predict()
  * 아이디, 아이템 아이디를 입력하면 
    - 추천 예측 평점을 포함한 정보를 반환
    - 기존 평점 정보(r_ui)는 선택 사항

In [12]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)

pred = algo.predict(uid, iid)

print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


# # 결론 
  * test() 메서드는 입력 데이터 세트의 모든 사용자와 아이템 아이디에 대해서 predict()를 반복적으로 수행한 결과

# # 성능 평가

In [13]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

# # [주요 모듈]

# # 데이터 불러오기

In [15]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Learning/머신러닝/권철민-파이썬머신러닝완벽가이드(위키북스)/res/Movie List _latest_small/ratings.csv')

# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.

ratings.to_csv('./ratings_noh.csv',
               index=False,
               header=False)

In [17]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [18]:
ratings_noh = pd.read_csv('/content/ratings_noh.csv')
ratings_noh

,1,1.1,4.0,964982703
0,1,3,4.0,964981247
1,1,6,4.0,964982224
2,1,47,5.0,964983815
3,1,50,5.0,964982931
4,1,70,3.0,964982400
...,...,...,...,...
100830,610,166534,4.0,1493848402
100831,610,168248,5.0,1493850091
100832,610,168250,5.0,1494273047
100833,610,168252,5.0,1493846352


In [16]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp',
                sep=',',
                rating_scale=(0.5, 5)
                )

data=Dataset.load_from_file('./ratings_noh.csv',
                            reader=reader)

In [21]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [22]:
# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

## ## DataFrame으로 데이터 로드

In [24]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Learning/머신러닝/권철민-파이썬머신러닝완벽가이드(위키북스)/res/Movie List _latest_small/ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [25]:
data

In [26]:
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

# # 베이스라인 평점
  * 각 개인의 성향에 따라 같은 아이템도 평가가 다를 수 있다.
  * 개인의 성향을 반영해 아이템 평가에 편향성(bias) 요소를 반영하여 부과하는 것을 베이스라인 평점(baseline rating)이라고 한다.
  * 베이스라인 평점 = 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수
    - 전체 평균 평점 = 모든 사용자의 아이템에 대한 평점을 평균한 값
    - 사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점
    - 아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균 평점

# # 교차 검증

In [27]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Learning/머신러닝/권철민-파이썬머신러닝완벽가이드(위키북스)/res/Movie List _latest_small/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8602  0.8752  0.8716  0.8759  0.8822  0.8730  0.0073  
MAE (testset)     0.6657  0.6710  0.6701  0.6705  0.6792  0.6713  0.0044  
Fit time          5.50    5.03    4.99    4.98    5.05    5.11    0.20    
Test time         0.16    0.29    0.15    0.29    0.15    0.21    0.07    


{'fit_time': (5.50170111656189,
  5.025991678237915,
  4.9852190017700195,
  4.978484392166138,
  5.045786142349243),
 'test_mae': array([0.66571343, 0.67095634, 0.67006614, 0.67052894, 0.67920853]),
 'test_rmse': array([0.86018989, 0.8751598 , 0.8715718 , 0.87592034, 0.88219744]),
 'test_time': (0.16263031959533691,
  0.28772497177124023,
  0.15479278564453125,
  0.29474592208862305,
  0.14988327026367188)}

# # 하이퍼 파라미터 튜닝

In [28]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8770191094580769
{'n_epochs': 20, 'n_factors': 50}


# # Surprise 를 이용한 개인화 영화추천 시스템 구축
  * 아직 관람하지 않은 영화 중에서 개인 취향에 가장 적절한 영화 추천
  * 훈련할 때 train, test로 나누지 않고 전체 데이터로 훈련

## ## train_test_split( )으로 분리하지 않았을 때 
  * 데이터 세트에 fit( )를 하면 오류가 발생합니다.
  * 왜냐하면 surprise 내부에서 사용하는 Trainset 클래스 객체로 변환하지 않았기 때문

In [29]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: ignored

## ## DatasetAutoFolds 로 오류 해결

In [30]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))

# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='./ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

## ## 학습

In [31]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

## userid = 9 사용자의 movieId  42 영화 평점 예측

In [32]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Learning/머신러닝/권철민-파이썬머신러닝완벽가이드(위키북스)/res/Movie List _latest_small/movies.csv')

In [33]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [34]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


### ### 평점을 매긴 적 있는지 확인

In [36]:
# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


### ### 예측

In [37]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


## ## 보지 않은 영화 전체 평점 예측

### ### 보지 않은 영화 추출

In [42]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    
    print('# 평점 매긴 영화수:',len(seen_movies), '\n'
          '# 추천대상(안 본) 영화수:',len(unseen_movies), '\n'
          '# 전체 영화수:',len(total_movies), '\n'
          )
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

# 평점 매긴 영화수: 46 
# 추천대상(안 본) 영화수: 9696 
# 전체 영화수: 9742 



### ### 예측

In [43]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

# 평점 매긴 영화수: 46 
# 추천대상(안 본) 영화수: 9696 
# 전체 영화수: 9742 

##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
